In [1]:
#!sudo apt-get install python-pip python-dev

原文以及repo:[TensorFlow Wide & Deep Learning Tutorial (TF 1.12版本)](https://github.com/baidu-research/tensorflow-allreduce/blob/master/tensorflow/docs_src/tutorials/wide_and_deep.md), 还有[chrominium版本 (TF 2.0+版本)](https://chromium.googlesource.com/external/github.com/tensorflow/tensorflow/+/r0.10/tensorflow/g3doc/tutorials/wide_and_deep/index.md)

> 这篇Tutorial的目标主要是怎么使用`DNNLinearCombinedClassifier`这个High level API，我们需要学习的主要是如何处理categorical feature以及numeric feature，还有**feature crossing**.

In the previous [TensorFlow Linear Model Tutorial](https://chromium.googlesource.com/external/github.com/tensorflow/tensorflow/+/r0.10/tensorflow/g3doc/tutorials/wide), we trained a logistic regression model to predict the probability that the individual has an annual income of over 50,000 dollars using the [Census Income Dataset](https://archive.ics.uci.edu/ml/datasets/Census+Income). TensorFlow is great for training deep neural networks too, and you might be thinking which one you should choose—Well, why not both? Would it be possible to combine the strengths of both in one model?

In this tutorial, we'll introduce how to use the tf.estimator API to jointly train a wide linear model and a deep feed-forward neural network. This approach combines the strengths of memorization and generalization. It's useful for generic large-scale regression and classification problems with sparse input features (e.g. categorical features with a large number of possible feature values). If you're interested in learning more about how Wide & Deep Learning works, please check out our [research paper](http://arxiv.org/abs/1606.07792).

![](https://1.bp.blogspot.com/-Dw1mB9am1l8/V3MgtOzp3uI/AAAAAAAABGs/mP-3nZQCjWwdk6qCa5WraSpK8A7rSPj3ACLcB/s640/image04.png "Wide'n'Deep")

The figure above shows a comparison of a wide model (logistic regression with sparse features and transformations), a deep model (feed-forward neural network with an embedding layer and several hidden layers), and a Wide & Deep model (joint training of both). At a high level, there are only 3 steps to configure a wide, deep, or Wide & Deep model using the TF.Learn API:

1. Select features for the **WIDE** part: Choose the `sparse base columns` and `crossed columns` you want to use.
2. Select features for the **DEEP** part: Choose the `continuous columns`, the `embedding dimension for each categorical column`, and the hidden layer sizes.
3. Put them all together in a Wide & Deep model (`DNNLinearCombinedClassifier`).

And that‘s it! Let’s go through a simple example.

原代码链接已失效，新链接见此处:[wide_n_deep_tutorial.py](https://github.com/baidu-research/tensorflow-allreduce/blob/master/tensorflow/examples/learn/wide_n_deep_tutorial.py)

In [2]:
!python wide_n_deep_tutorial.py --model_type=wide_n_deep

python3: can't open file 'wide_n_deep_tutorial.py': [Errno 2] No such file or directory


注意，这篇tutorial有几个坑：
1. python为2.7环境  
2. TensorFlow为1.12.0版本

所以一上来就要给TF降级：

In [3]:
#!pip install tensorflow==1.14.0

In [4]:
import tensorflow as tf
print(tf.__version__)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

1.14.0


/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

# A Glance of Data

In [5]:
import pandas as pd
import urllib.request
import tempfile

# Define the column names for the data sets.
COLUMNS = ["age", "workclass", "fnlwgt", "education", "education_num",
  "marital_status", "occupation", "relationship", "race", "gender",
  "capital_gain", "capital_loss", "hours_per_week", "native_country", "income_bracket"]
LABEL_COLUMN = 'label'
CATEGORICAL_COLUMNS = ["workclass", "education", "marital_status", "occupation",
                       "relationship", "race", "gender", "native_country"]
CONTINUOUS_COLUMNS = ["age", "education_num", "capital_gain", "capital_loss",
                      "hours_per_week"]

# Download the training and test data to temporary files.
# Alternatively, you can download them yourself and change train_file and
# test_file to your own paths.
train_file = tempfile.NamedTemporaryFile()
test_file = tempfile.NamedTemporaryFile()
urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", train_file.name)
urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test", test_file.name)

# Read the training and test data sets into Pandas dataframe.
df_train = pd.read_csv(train_file, names=COLUMNS, skipinitialspace=True)
df_test = pd.read_csv(test_file, names=COLUMNS, skipinitialspace=True, skiprows=1)
df_train[LABEL_COLUMN] = (df_train['income_bracket'].apply(lambda x: '>50K' in x)).astype(int)
df_test[LABEL_COLUMN] = (df_test['income_bracket'].apply(lambda x: '>50K' in x)).astype(int)

In [6]:
df_train.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,0


In [7]:
df_train.education.unique()

array(['Bachelors', 'HS-grad', '11th', 'Masters', '9th', 'Some-college',
       'Assoc-acdm', 'Assoc-voc', '7th-8th', 'Doctorate', 'Prof-school',
       '5th-6th', '10th', '1st-4th', 'Preschool', '12th'], dtype=object)

# Define Base Feature Columns

First, let's define the base `categorical` and `continuous` feature columns that we'll use. **These base columns will be the building blocks used by both the wide part and the deep part of the model**.

In [8]:
import tensorflow as tf

#  Continuous base columns.
## bucketized cols
age = tf.contrib.layers.real_valued_column("age")
education_num = tf.contrib.layers.real_valued_column("education_num")
capital_gain = tf.contrib.layers.real_valued_column("capital_gain")
capital_loss = tf.contrib.layers.real_valued_column("capital_loss")
hours_per_week = tf.contrib.layers.real_valued_column("hours_per_week")


#  Categorical base columns.
## 类别较少的categorcial feature: sparse_column_with_keys()
gender = tf.contrib.layers.sparse_column_with_keys(column_name="gender", keys=["female", "male"])
race = tf.contrib.layers.sparse_column_with_keys(column_name="race", keys=["Amer-Indian-Eskimo", 
                                                                           "Asian-Pac-Islander", 
                                                                           "Black", 
                                                                           "Other", 
                                                                           "White"]
                                                 )
## 类别较多的categorcial feature: sparse_column_with_hash_bucket()
education = tf.contrib.layers.sparse_column_with_hash_bucket("education", hash_bucket_size=1000)
marital_status = tf.contrib.layers.sparse_column_with_hash_bucket("marital_status", hash_bucket_size=100)
relationship = tf.contrib.layers.sparse_column_with_hash_bucket("relationship", hash_bucket_size=100)
workclass = tf.contrib.layers.sparse_column_with_hash_bucket("workclass", hash_bucket_size=100)
occupation = tf.contrib.layers.sparse_column_with_hash_bucket("occupation", hash_bucket_size=1000)
native_country = tf.contrib.layers.sparse_column_with_hash_bucket("native_country", hash_bucket_size=1000)

# Transformations.
age_buckets = tf.contrib.layers.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [9]:
# # To show an example of hashing:
# occupation = tf.feature_column.categorical_column_with_hash_bucket(
#     "occupation", hash_bucket_size=1000)
# native_country = tf.feature_column.categorical_column_with_hash_bucket(
#     "native_country", hash_bucket_size=1000)

# The Wide Model: Linear Model with Crossed Feature Columns
The wide model is a linear model with a wide set of sparse and crossed feature columns:



In [10]:
wide_columns = [
  gender, native_country, education, occupation, workclass, marital_status, relationship, age_buckets,
  tf.contrib.layers.crossed_column([education, occupation], hash_bucket_size=int(1e4)),               # 将education和occupation两者做feature crossing
  tf.contrib.layers.crossed_column([native_country, occupation], hash_bucket_size=int(1e4)),
  tf.contrib.layers.crossed_column([age_buckets, race, occupation], hash_bucket_size=int(1e6))]      # 将age_buckets， race和occupation三者做feature crossing

In [11]:
wide_columns # 可以看到全是各种feature column

[_SparseColumnKeys(column_name='gender', is_integerized=False, bucket_size=None, lookup_config=_SparseIdLookupConfig(vocabulary_file=None, keys=('female', 'male'), num_oov_buckets=0, vocab_size=2, default_value=-1), combiner='sum', dtype=tf.string),
 _SparseColumnHashed(column_name='native_country', is_integerized=False, bucket_size=1000, lookup_config=None, combiner='sum', dtype=tf.string),
 _SparseColumnHashed(column_name='education', is_integerized=False, bucket_size=1000, lookup_config=None, combiner='sum', dtype=tf.string),
 _SparseColumnHashed(column_name='occupation', is_integerized=False, bucket_size=1000, lookup_config=None, combiner='sum', dtype=tf.string),
 _SparseColumnHashed(column_name='workclass', is_integerized=False, bucket_size=100, lookup_config=None, combiner='sum', dtype=tf.string),
 _SparseColumnHashed(column_name='marital_status', is_integerized=False, bucket_size=100, lookup_config=None, combiner='sum', dtype=tf.string),
 _SparseColumnHashed(column_name='relatio

Wide models with crossed feature columns can memorize sparse interactions between features effectively. That being said, one limitation of crossed feature columns is that they do not generalize to feature combinations that have not appeared in the training data. Let's add a deep model with embeddings to fix that.

# The Deep Model: Neural Network with Embeddings
The deep model is a feed-forward neural network, as shown in the previous figure. **Each of the sparse, high-dimensional `categorical features` are first converted into a low-dimensional and dense real-valued vector, often referred to as an embedding vector**. These low-dimensional dense embedding vectors are concatenated with the continuous features, and then fed into the hidden layers of a neural network in the forward pass. The embedding values are initialized randomly, and are trained along with all other model parameters to minimize the training loss. If you're interested in learning more about embeddings, check out the TensorFlow tutorial on [Vector Representations of Words](https://www.tensorflow.org/versions/r0.9/tutorials/word2vec/index.html), or Word Embedding on Wikipedia.

Another way to represent categorical columns to feed into a neural network is via a multi-hot representation. This is often appropriate for categorical columns with only a few possible values. E.g. for the gender column, "Male" can be represented as [1, 0] and "Female" as [0, 1]. This is a fixed representation, whereas embeddings are more flexible and calculated at training time.

We'll configure the embeddings for the categorical columns using embedding_column, and concatenate them with the continuous columns. We also use indicator_column to create multi-hot representation of some categorical columns.

In [12]:
# 对categorical feature的政策就是全部要embedding
deep_columns = [
  tf.contrib.layers.embedding_column(workclass, dimension=8),   # dimension越大准确性越高
  tf.contrib.layers.embedding_column(education, dimension=8),
  tf.contrib.layers.embedding_column(marital_status, dimension=8),
  tf.contrib.layers.embedding_column(gender, dimension=8),
  tf.contrib.layers.embedding_column(relationship, dimension=8),
  tf.contrib.layers.embedding_column(race, dimension=8),
  tf.contrib.layers.embedding_column(native_country, dimension=8),
  tf.contrib.layers.embedding_column(occupation, dimension=8),
  age, education_num, capital_gain, capital_loss, hours_per_week
  ]


# TF 2.0+版本差异还挺大
#  deep_columns = [
#     tf.feature_column.indicator_column(workclass),
#     tf.feature_column.indicator_column(education),
#     tf.feature_column.indicator_column(gender),
#     tf.feature_column.indicator_column(relationship),
#     # To show an example of embedding
#     tf.feature_column.embedding_column(native_country, dimension=8),
#     tf.feature_column.embedding_column(occupation, dimension=8),
#     age,
#     education_num,
#     capital_gain,
#     capital_loss,
#     hours_per_week,
# ] 

The higher the dimension of the embedding is, the more degrees of freedom the model will have to learn the representations of the features. For simplicity, we set the dimension to 8 for all feature columns here. **Empirically, a more informed decision for the number of dimensions is to start with a value on the order of $(\log_2(n))$ or $(k\sqrt[4]n)$, where $(n)$ is the number of unique features in a feature column and $(k)$ is a small constant (usually smaller than 10)**. 

Through dense embeddings, deep models can generalize better and make predictions on feature pairs that were previously unseen in the training data. (**Deep model优点是通过dense embedding, 很多没有pair过的feature现在也能找到相关性了**) However, it is difficult to learn effective low-dimensional representations for feature columns when the underlying interaction matrix between two feature columns is sparse and high-rank. In such cases, the interaction between most feature pairs should be zero except a few, but dense embeddings will lead to nonzero predictions for all feature pairs, and thus can over-generalize. (**Deep model缺点是学到的很多相关性太泛泛**)On the other hand, linear models with crossed features can memorize these “exception rules” effectively with fewer model parameters.

Now, let's see how to jointly train wide and deep models and allow them to complement each other’s strengths and weaknesses.

## Feature 汇总
- Wide part：
 - gender 
 - native_country
 - education
 - occupation
 - workclass
 - marital_status
 - relationship 
 - age_buckets,
 - education X occupation 
 - native_country X occupation
 - age_buckets X race X occupation
- Deep part:
 - workclass 
 - education
 - marital_status
 - gender
 - relationship
 - race
 - native_country
 - occupation
 - age
 - education_num 
 - capital_gain 
 - capital_loss,
 - hours_per_week


#Combining Wide and Deep Models into One
**The wide models and deep models are combined by summing up their final output log odds as the prediction, then feeding the prediction to a logistic loss function**. All the graph definition and variable allocations have already been handled for you under the hood, so you simply need to create a `DNNLinearCombinedClassifier`:

In [13]:
import tempfile
model_dir = tempfile.mkdtemp()

# 只需要提供wide和deep column的数据，剩下的DNNLinearCombinedClassifier()来处理
m = tf.contrib.learn.DNNLinearCombinedClassifier(
    model_dir=model_dir,
    linear_feature_columns=wide_columns,
    dnn_feature_columns=deep_columns,
    dnn_hidden_units=[100, 50])

Instructions for updating:
Please set fix_global_step_increment_bug=True and update training steps in your pipeline. See pydoc for details.
Instructions for updating:
Please switch to tf.contrib.estimator.*_head.
Instructions for updating:
Please replace uses of any Estimator from tf.contrib.learn with an Estimator from tf.estimator.*
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f048e43b400>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_sum

#Training and Evaluating The Model
Before we train the model, let's read in the Census dataset as we did in the[TensorFlow Linear Model tutorial](https://chromium.googlesource.com/external/github.com/tensorflow/tensorflow/+/r0.10/tensorflow/g3doc/tutorials/wide). The code for input data processing is provided here again for your convenience:

注：这个模型厉害的地方在于，可能线下训练的Top 1的AUC看起来一般般，但是上线之后推荐多个结果的TopN整体效果很好，会包含很多新的有意思的app。

In [14]:
import pandas as pd
import urllib.request

# Define the column names for the data sets.
COLUMNS = ["age", "workclass", "fnlwgt", "education", "education_num",
  "marital_status", "occupation", "relationship", "race", "gender",
  "capital_gain", "capital_loss", "hours_per_week", "native_country", "income_bracket"]
LABEL_COLUMN = 'label'
CATEGORICAL_COLUMNS = ["workclass", "education", "marital_status", "occupation",
                       "relationship", "race", "gender", "native_country"]
CONTINUOUS_COLUMNS = ["age", "education_num", "capital_gain", "capital_loss",
                      "hours_per_week"]

# Download the training and test data to temporary files.
# Alternatively, you can download them yourself and change train_file and
# test_file to your own paths.
train_file = tempfile.NamedTemporaryFile()
test_file = tempfile.NamedTemporaryFile()
urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", train_file.name)
urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test", test_file.name)

# Read the training and test data sets into Pandas dataframe.
df_train = pd.read_csv(train_file, names=COLUMNS, skipinitialspace=True)
df_test = pd.read_csv(test_file, names=COLUMNS, skipinitialspace=True, skiprows=1)
df_train[LABEL_COLUMN] = (df_train['income_bracket'].apply(lambda x: '>50K' in x)).astype(int)
df_test[LABEL_COLUMN] = (df_test['income_bracket'].apply(lambda x: '>50K' in x)).astype(int)

In [15]:
df_train.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,0


In [16]:
def input_fn(df):
  # Creates a dictionary mapping from each continuous feature column name (k) to
  # the values of that column stored in a constant Tensor.
  continuous_cols = {k: tf.constant(df[k].values)
                     for k in CONTINUOUS_COLUMNS}

  # Creates a dictionary mapping from each categorical feature column name (k)
  # to the values of that column stored in a tf.SparseTensor.
  categorical_cols = {k: tf.SparseTensor(
      indices=[[i, 0] for i in range(df[k].size)],
      values=df[k].values,
      dense_shape=[df[k].size, 1]) for k in CATEGORICAL_COLUMNS
      }

  # Merges the two dictionaries into one.
  feature_cols = dict(continuous_cols.items() | categorical_cols.items())
  
  # Converts the label column into a constant Tensor.
  label = tf.constant(df[LABEL_COLUMN].values)

  # Returns the feature columns and the label.
  print(continuous_cols, feature_cols, label)
  
  return feature_cols, label

def train_input_fn():
  return input_fn(df_train)

def eval_input_fn():
  return input_fn(df_test)

In [17]:
input_fn(df_train)

{'age': <tf.Tensor 'Const:0' shape=(32561,) dtype=int64>, 'education_num': <tf.Tensor 'Const_1:0' shape=(32561,) dtype=int64>, 'capital_gain': <tf.Tensor 'Const_2:0' shape=(32561,) dtype=int64>, 'capital_loss': <tf.Tensor 'Const_3:0' shape=(32561,) dtype=int64>, 'hours_per_week': <tf.Tensor 'Const_4:0' shape=(32561,) dtype=int64>} {'native_country': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f048e476f28>, 'age': <tf.Tensor 'Const:0' shape=(32561,) dtype=int64>, 'capital_gain': <tf.Tensor 'Const_2:0' shape=(32561,) dtype=int64>, 'gender': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f04833c0da0>, 'hours_per_week': <tf.Tensor 'Const_4:0' shape=(32561,) dtype=int64>, 'marital_status': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f04aa0e9198>, 'relationship': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f048e43b3c8>, 'education_num': <tf.Tensor 'Const_1:0' shape=(32561,) dtype=int64>, 'capital

({'age': <tf.Tensor 'Const:0' shape=(32561,) dtype=int64>,
  'capital_gain': <tf.Tensor 'Const_2:0' shape=(32561,) dtype=int64>,
  'capital_loss': <tf.Tensor 'Const_3:0' shape=(32561,) dtype=int64>,
  'education': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x7f048389cc88>,
  'education_num': <tf.Tensor 'Const_1:0' shape=(32561,) dtype=int64>,
  'gender': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x7f04833c0da0>,
  'hours_per_week': <tf.Tensor 'Const_4:0' shape=(32561,) dtype=int64>,
  'marital_status': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x7f04aa0e9198>,
  'native_country': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x7f048e476f28>,
  'occupation': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x7f048341d0b8>,
  'race': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x7f0482d81c18>,
  'relationship': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x7f048e43b3c8>,
  'workclass': 

After reading in the data, you can train and evaluate the model

In [18]:
m.fit(input_fn=train_input_fn, steps=200)

results = m.evaluate(input_fn=eval_input_fn, steps=1)
# for key in sorted(results):
#     print ("%s: %s" % (key, results[key]))

{'age': <tf.Tensor 'Const:0' shape=(32561,) dtype=int64>, 'education_num': <tf.Tensor 'Const_1:0' shape=(32561,) dtype=int64>, 'capital_gain': <tf.Tensor 'Const_2:0' shape=(32561,) dtype=int64>, 'capital_loss': <tf.Tensor 'Const_3:0' shape=(32561,) dtype=int64>, 'hours_per_week': <tf.Tensor 'Const_4:0' shape=(32561,) dtype=int64>} {'race': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f0482412668>, 'education_num': <tf.Tensor 'Const_1:0' shape=(32561,) dtype=int64>, 'gender': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f0482412b70>, 'native_country': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f04824121d0>, 'hours_per_week': <tf.Tensor 'Const_4:0' shape=(32561,) dtype=int64>, 'marital_status': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f04aa0e88d0>, 'capital_loss': <tf.Tensor 'Const_3:0' shape=(32561,) dtype=int64>, 'age': <tf.Tensor 'Const:0' shape=(32561,) dtype=int64>, 'capital_gain': 

In [19]:
results

{'accuracy': 0.8227382,
 'accuracy/baseline_label_mean': 0.23622628,
 'accuracy/threshold_0.500000_mean': 0.8227382,
 'auc': 0.8022439,
 'auc_precision_recall': 0.6209644,
 'global_step': 202,
 'labels/actual_label_mean': 0.23622628,
 'labels/prediction_mean': 0.22765823,
 'loss': 0.92337215,
 'precision/positive_threshold_0.500000_mean': 0.67531043,
 'recall/positive_threshold_0.500000_mean': 0.48075923}

The first line of the output should be something like `accuracy: 0.84429705`. We can see that the accuracy was improved from about 83.6% using a wide-only linear model to about 84.4% using a Wide & Deep model. If you'd like to see a working end-to-end example, you can download our example code.

Note that this tutorial is just a quick example on a small dataset to get you familiar with the API. **Wide & Deep Learning will be even more powerful if you try it on a large dataset with many sparse feature columns that have a large number of possible feature values**. Again, feel free to take a look at our research paper for more ideas about how to apply Wide & Deep Learning in real-world large-scale machine learning problems.